In [22]:
import os
import glob
import shutil
import pandas as pd
import geopandas as gpd

In [23]:
# input sample paths
# the order is importent
BASE_PATHS = [
    '../example_3',
    '../example_2',
    '../example_1'
]

# input paths inside of an each sample path
INPUT_RESULT_FOLDER = 'output/analyser'
INPUT_DB = 'db/database.gpkg'

# output folder - will include comparisons
OUTPUT_CSV_FOLDER = '../csv'

In [24]:
def clean_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)     

paths = BASE_PATHS
names = list(map(lambda p: os.path.basename(p), paths))

clean_folder(OUTPUT_CSV_FOLDER)

In [25]:
# distance between points 
db_file = os.path.join(paths[0], INPUT_DB)
profiles = gpd.read_file(db_file, layer='profiles')
points_distance = round(profiles.iloc[0].geometry.distance(profiles.iloc[1].geometry), 3)

In [26]:
def get_results_diff(first, second):
    merged = first.merge(second, left_on='profile_id', right_on='profile_id', how='left')
    diff = pd.DataFrame({
        'profile_id': merged.profile_id,
        'zero_position': (merged.last_zero_id_y - merged.last_zero_id_x) * points_distance,
        'zero_elevation': merged.last_zero_elevation_y - merged.last_zero_elevation_x,

        'bottom_position': (merged.bottom_id_y - merged.bottom_id_x) * points_distance,
        'bottom_elevation': merged.bottom_elevation_y - merged.bottom_elevation_x,

        'top_position': (merged.top_id_y - merged.top_id_x) * points_distance,
        'top_elevation': merged.top_elevation_y - merged.top_elevation_x,

        'to_bottom_distance': merged.to_bottom_distance_y - merged.to_bottom_distance_x,
        'to_bottom_slope': merged.to_bottom_slope_y - merged.to_bottom_slope_x,
        'to_bottom_volume': merged.to_bottom_volume_y - merged.to_bottom_volume_x,

        'to_top_distance': merged.to_top_distance_y - merged.to_top_distance_x,
        'to_top_slope': merged.to_top_slope_y - merged.to_top_slope_x,
        'to_top_volume': merged.to_top_volume_y - merged.to_top_volume_x  
    })
    return diff

In [27]:
# analyser results
samples = []

for path in paths:
    results_file = glob.glob(os.path.join(path, INPUT_RESULT_FOLDER, '*.csv'))
    if len(results_file) == 0:
        break
    res = pd.read_csv(results_file[0], encoding='utf-8', sep=',', skipinitialspace=True)
    samples.append(res)    

In [28]:
diffs = []

for idx, sample in enumerate(samples[:-1]):
    diffs.append(get_results_diff(samples[idx], samples[idx+1]))

In [30]:
fields = [
    'zero_position',
    'zero_elevation',
    'bottom_position',
    'bottom_elevation',
    'top_position',
    'top_elevation',
    'to_bottom_distance',
    'to_bottom_slope',
    'to_bottom_volume',
    'to_top_distance',
    'to_top_slope',
    'to_top_volume'
]

for field in fields:
    field_diffs = pd.DataFrame({
        'profile_id': diffs[0].profile_id
    })
    for diff_idx, diff in enumerate(diffs):
        field_diffs = field_diffs.merge(diff[['profile_id', field]], left_on='profile_id', right_on='profile_id', how='left')
        field_diffs.rename(columns = {field: f'{names[diff_idx]}_{names[diff_idx+1]}'}, inplace = True)
    
    field_diffs.to_csv(os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv'), encoding='utf-8', sep=',')